In [122]:
# Snowpark for Python
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, udf
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import StructType, StructField, DoubleType, StringType, DecimalType
from snowflake.snowpark.version import VERSION

# Snowpark Machine Learning
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.registry import registry

# Data Science Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Miscellaneous
import json
import warnings
warnings.simplefilter('ignore')  # Suppress warnings


# Establish a connection to a Snowflake

In [123]:
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : PETERSIMMONS1991
Role                        : "ACCOUNTADMIN"
Database                    : "ML_HOL_DB"
Schema                      : "ML_HOL_SCHEMA"
Warehouse                   : "ML_HOL_WH"
Snowflake version           : 8.6.2
Snowpark for Python version : 1.11.1


# Data Preprocessing

In [124]:
# Show the file before loading
session.sql("LS @Loan_Data;").show()

---------------------------------------------------------------------------------------------------------------
|"name"                           |"size"  |"md5"                             |"last_modified"                |
---------------------------------------------------------------------------------------------------------------
|loan_data/loandata_modified.csv  |764064  |850a631ef1eef027ee9a3211da3ece4c  |Thu, 15 Feb 2024 03:39:11 GMT  |
---------------------------------------------------------------------------------------------------------------



In [125]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
# We can now infer schema from CSV files.
loandata_df = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@Loan_Data")


In [126]:
loandata_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"credit_policy"  |"purpose"           |"int_rate"  |"installment"  |"log_annual_inc"  |"dti"  |"fico"  |"days_with_cr_line"  |"revol_bal"  |"revol_util"  |"inq_last_6mths"  |"delinq_2yrs"  |"pub_rec"  |"not_fully_paid"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1                |debt_consolidation  |0.1189      |829.10         |11.350406540      |19.48  |737     |5639.9583330         |28854        |52.10         |0                 |0              |0          |0                 |
|1                |credit_card         |0.1071      |228.22         |11.082142550      |14.29  |707     |276

In [127]:
# Look at descriptive stats on the DataFrame
loandata_df.describe().show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"credit_policy"      |"purpose"       |"int_rate"            |"installment"       |"log_annual_inc"    |"dti"              |"fico"             |"days_with_cr_line"  |"revol_bal"        |"revol_util"       |"inq_last_6mths"    |"delinq_2yrs"      |"pub_rec"           |"not_fully_paid"     |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |9578.0               |9578            |9578.0                |9578.0       

In [128]:
loandata_df.columns

['"credit_policy"',
 '"purpose"',
 '"int_rate"',
 '"installment"',
 '"log_annual_inc"',
 '"dti"',
 '"fico"',
 '"days_with_cr_line"',
 '"revol_bal"',
 '"revol_util"',
 '"inq_last_6mths"',
 '"delinq_2yrs"',
 '"pub_rec"',
 '"not_fully_paid"']

In [129]:
# Force headers to uppercase
for colname in loandata_df.columns:
    new_colname = str.upper(colname)
    loandata_df = loandata_df.with_column_renamed(colname, new_colname)

loandata_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CREDIT_POLICY"  |"PURPOSE"           |"INT_RATE"  |"INSTALLMENT"  |"LOG_ANNUAL_INC"  |"DTI"  |"FICO"  |"DAYS_WITH_CR_LINE"  |"REVOL_BAL"  |"REVOL_UTIL"  |"INQ_LAST_6MTHS"  |"DELINQ_2YRS"  |"PUB_REC"  |"NOT_FULLY_PAID"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1                |debt_consolidation  |0.1189      |829.10         |11.350406540      |19.48  |737     |5639.9583330         |28854        |52.10         |0                 |0              |0          |0                 |
|1                |credit_card         |0.1071      |228.22         |11.082142550      |14.29  |707     |276

In [130]:
# Define a function to clean a column by replacing non-alphanumeric characters with underscores and converting to uppercase.

def fix_values(columnn):
    return F.upper(F.regexp_replace(F.col(columnn), '[^a-zA-Z0-9]+', '_'))

# Apply the 'fix_values' function to the 'PURPOSE' column of the 'loandata_df' DataFrame to clean its values.
for col in ["PURPOSE"]:
    loandata_df = loandata_df.with_column(col, fix_values(col))
    
# Display the DataFrame to review the changes made to the 'PURPOSE' column.
loandata_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CREDIT_POLICY"  |"INT_RATE"  |"INSTALLMENT"  |"LOG_ANNUAL_INC"  |"DTI"  |"FICO"  |"DAYS_WITH_CR_LINE"  |"REVOL_BAL"  |"REVOL_UTIL"  |"INQ_LAST_6MTHS"  |"DELINQ_2YRS"  |"PUB_REC"  |"NOT_FULLY_PAID"  |"PURPOSE"           |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1                |0.1189      |829.10         |11.350406540      |19.48  |737     |5639.9583330         |28854        |52.10         |0                 |0              |0          |0                 |DEBT_CONSOLIDATION  |
|1                |0.1071      |228.22         |11.082142550      |14.29  |707     |2760.0000000         |33

In [131]:
list(loandata_df.schema)

[StructField('CREDIT_POLICY', LongType(), nullable=True),
 StructField('INT_RATE', DecimalType(5, 4), nullable=True),
 StructField('INSTALLMENT', DecimalType(5, 2), nullable=True),
 StructField('LOG_ANNUAL_INC', DecimalType(11, 9), nullable=True),
 StructField('DTI', DecimalType(4, 2), nullable=True),
 StructField('FICO', LongType(), nullable=True),
 StructField('DAYS_WITH_CR_LINE', DecimalType(12, 7), nullable=True),
 StructField('REVOL_BAL', LongType(), nullable=True),
 StructField('REVOL_UTIL', DecimalType(5, 2), nullable=True),
 StructField('INQ_LAST_6MTHS', LongType(), nullable=True),
 StructField('DELINQ_2YRS', LongType(), nullable=True),
 StructField('PUB_REC', LongType(), nullable=True),
 StructField('NOT_FULLY_PAID', LongType(), nullable=True),
 StructField('PURPOSE', StringType(16777216), nullable=True)]

In [132]:
# Loop through a list of column names and cast each column to DoubleType in the 'loandata_df' DataFrame.
for colname in ["CREDIT_POLICY", "INT_RATE", "INSTALLMENT", "LOG_ANNUAL_INC", "DTI", "FICO","DAYS_WITH_CR_LINE","REVOL_BAL","REVOL_UTIL","INQ_LAST_6MTHS","DELINQ_2YRS","PUB_REC","NOT_FULLY_PAID"]:
    loandata_df = loandata_df.with_column(colname, loandata_df[colname].cast(DoubleType()))
list(loandata_df.schema)

[StructField('PURPOSE', StringType(16777216), nullable=True),
 StructField('CREDIT_POLICY', DoubleType(), nullable=True),
 StructField('INT_RATE', DoubleType(), nullable=True),
 StructField('INSTALLMENT', DoubleType(), nullable=True),
 StructField('LOG_ANNUAL_INC', DoubleType(), nullable=True),
 StructField('DTI', DoubleType(), nullable=True),
 StructField('FICO', DoubleType(), nullable=True),
 StructField('DAYS_WITH_CR_LINE', DoubleType(), nullable=True),
 StructField('REVOL_BAL', DoubleType(), nullable=True),
 StructField('REVOL_UTIL', DoubleType(), nullable=True),
 StructField('INQ_LAST_6MTHS', DoubleType(), nullable=True),
 StructField('DELINQ_2YRS', DoubleType(), nullable=True),
 StructField('PUB_REC', DoubleType(), nullable=True),
 StructField('NOT_FULLY_PAID', DoubleType(), nullable=True)]

INT_RATE (Interest Rate)
INSTALLMENT
LOG_ANNUAL_INC (Logarithm of Annual Income)
DTI (Debt-to-Income Ratio)
FICO (FICO Credit Score)
DAYS_WITH_CR_LINE (Days with Credit Line)
REVOL_BAL (Revolving Balance)
REVOL_UTIL (Revolving Line Utilization Rate)
INQ_LAST_6MTHS (Inquiries in Last 6 Months)
DELINQ_2YRS (Delinquencies in the Past 2 Years)
PUB_REC (Public Records)

In [133]:
loandata_df.write.mode('overwrite').save_as_table('loandata_trial123')

In [134]:
from snowflake.snowpark.functions import col
unique_PURPOSE=loandata_df.select(col("PURPOSE")).distinct()
unique_PURPOSE.show()

----------------------
|"PURPOSE"           |
----------------------
|DEBT_CONSOLIDATION  |
|CREDIT_CARD         |
|ALL_OTHER           |
|HOME_IMPROVEMENT    |
|SMALL_BUSINESS      |
|MAJOR_PURCHASE      |
|EDUCATIONAL         |
----------------------



In [135]:
categories = {"PURPOSE": np.array(["DEBT_CONSOLIDATION", "CREDIT_CARD", "ALL_OTHER", "HOME_IMPROVEMENT", "SMALL_BUSINESS","MAJOR_PURCHASE","EDUCATIONAL"]) }

In [136]:
# Initialize an OrdinalEncoder to encode the 'PURPOSE' column according to the specified categories.
snowml_oe = snowml.OrdinalEncoder(input_cols=["PURPOSE"], output_cols=["PURPOSE_OE"], categories=categories)

# Fit the OrdinalEncoder to the loandata_df and transform it, encoding the 'PURPOSE' column.
ord_encoded_loandata_df = snowml_oe.fit(loandata_df).transform(loandata_df)


# Drop the original 'PURPOSE' column from the DataFrame after encoding.
# Rename the encoded column 'PURPOSE_OE' back to 'PURPOSE' for clarity in the final DataFrame.
ord_encoded_loandata_df=ord_encoded_loandata_df.drop("PURPOSE")
df_clean = ord_encoded_loandata_df.rename(col("PURPOSE_OE"), "PURPOSE")

In [137]:
df_clean.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PURPOSE"  |"CREDIT_POLICY"  |"INT_RATE"  |"INSTALLMENT"  |"LOG_ANNUAL_INC"  |"DTI"  |"FICO"  |"DAYS_WITH_CR_LINE"  |"REVOL_BAL"  |"REVOL_UTIL"  |"INQ_LAST_6MTHS"  |"DELINQ_2YRS"  |"PUB_REC"  |"NOT_FULLY_PAID"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.0        |0.0              |0.1829      |411.99         |11.46163217       |20.48  |652.0   |7207.0               |10705.0      |44.6          |7.0               |1.0            |1.0        |0.0               |
|0.0        |0.0              |0.1197      |166.0          |10.30895266       |20.76  |687.0   |4079.958333          |14536.0      |68.9        

# Handling class imbalance by over-sampling

In [138]:

from snowflake.snowpark.functions import count, lit

count_df = df_clean.group_by("NOT_FULLY_PAID").agg(count("*").alias("count"))

# Calculate the proportion of each category
total_count = df_clean.count()
proportion_df = count_df.with_column("proportion", count_df["count"] / lit(total_count))

# Show the result
proportion_df.show()




---------------------------------------------
|"NOT_FULLY_PAID"  |"COUNT"  |"PROPORTION"  |
---------------------------------------------
|0.0               |8045     |0.839946      |
|1.0               |1533     |0.160054      |
---------------------------------------------



In [139]:
# Handling class imbalance by over-sampling
df_pd=df_clean.to_pandas()

count_class_0, count_class_1 = df_pd['NOT_FULLY_PAID'].value_counts()
df_0 = df_pd[df_pd['NOT_FULLY_PAID'] == 0]
df_1 = df_pd[df_pd['NOT_FULLY_PAID'] == 1]
df_1_over = df_1.sample(count_class_0, replace=True)
df_test_over_pd = pd.concat([df_0, df_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over_pd['NOT_FULLY_PAID'].value_counts())

df_final=session.create_dataframe(df_test_over_pd) #pandas to snowpark


Random over-sampling:
0.0    8045
1.0    8045
Name: NOT_FULLY_PAID, dtype: int64


In [140]:
df_final.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PURPOSE"  |"CREDIT_POLICY"  |"INT_RATE"  |"INSTALLMENT"  |"LOG_ANNUAL_INC"  |"DTI"  |"FICO"  |"DAYS_WITH_CR_LINE"  |"REVOL_BAL"  |"REVOL_UTIL"  |"INQ_LAST_6MTHS"  |"DELINQ_2YRS"  |"PUB_REC"  |"NOT_FULLY_PAID"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.0        |0.0              |0.1829      |411.99         |11.46163217       |20.48  |652.0   |7207.0               |10705.0      |44.6          |7.0               |1.0            |1.0        |0.0               |
|0.0        |0.0              |0.1197      |166.0          |10.30895266       |20.76  |687.0   |4079.958333          |14536.0      |68.9        

# Hyperparameter tuning

In [141]:

# Initialize an XGBClassifier model, then split the dataset into training and testing sets.
# Set up and perform grid search with specified hyperparameters to find the best model configuration, focusing on accuracy.

from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.model_selection import GridSearchCV

from snowflake.snowpark.functions import col


# Define the model
model = XGBClassifier()
train_df, test_df = df_final.random_split(weights=[0.9, 0.1], seed=0)

# Specify hyperparameters to tune
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

grid_search=GridSearchCV(
    estimator=XGBClassifier(),
    param_grid=param_grid,
    n_jobs=-1,
    scoring="accuracy",
    input_cols=train_df.drop("NOT_FULLY_PAID").columns,
    label_cols="NOT_FULLY_PAID",
    output_cols="PREDICTION",
)

grid_search.fit(train_df)


<b>max_depth</b>: Determines the maximum depth of the trees. Deeper trees can model more complex patterns but might lead to overfitting. Values explored are 3, 4, and 5, aiming to find a balance between model complexity and generalization.
<br><br>
<b>learning_rate</b>: Controls the step size at each iteration while moving toward a minimum of a loss function. A lower rate requires more iterations but can achieve a more accurate model. Tested rates are 0.01, 0.1, and 0.2, to evaluate the trade-off between convergence speed and model performance.
<br><br>
<b>n_estimators</b>: Specifies the number of trees in the ensemble. More trees can lead to better performance but also increase computation time. The grid search tests 100, 200, and 300 trees to find an optimal number for the ensemble size.
<br><br>
<b>Grid Search</b>:<br>
Grid search is a brute-force method for hyperparameter optimization. It systematically creates and evaluates models for each combination of the parameter grid specified. This approach ensures that every possible combination is tested but can be computationally expensive, especially with large datasets and many parameters.
<br><br>
<b>Other Types of Hyperparameter Optimizations</b>:
<br><br>
<b>Random Search</b>: Samples parameter combinations randomly. This method is more efficient than grid search when dealing with a large number of hyperparameters.
<br><br>
<b>Bayesian Optimization</b>: Uses a probabilistic model to predict the performance of parameter combinations and selects new combinations to test based on past results, optimizing both exploration of the parameter space and exploitation of known good areas.
<br><br>
<b>Gradient-based Optimization</b>: Applies gradient descent or similar methods to optimize hyperparameters, particularly useful when the parameters are continuous.
<br><br>
<b>Evolutionary Algorithms</b>: Mimic the process of natural selection to iteratively select, mutate, and combine parameters to find optimal solutions over generations.
<br><br>
<b>Why Hyperparameter Optimization is Needed</b>:
Hyperparameter optimization is crucial for machine learning models to enhance their performance on a given task. The default parameters of a model might not be ideal for all types of data or problems. By tuning hyperparameters, we can significantly improve a model's accuracy, efficiency, and generalization ability to unseen data, ensuring the model is well-suited for its specific application.

In [142]:
# Use the best model found by grid search to make predictions on the test dataset and store the results in `predictions_df`.

predictions_df = grid_search.predict(test_df)

In [143]:
predictions_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CREDIT_POLICY"  |"REVOL_UTIL"  |"INQ_LAST_6MTHS"  |"FICO"  |"LOG_ANNUAL_INC"  |"DELINQ_2YRS"  |"PUB_REC"  |"DAYS_WITH_CR_LINE"  |"INSTALLMENT"  |"DTI"  |"NOT_FULLY_PAID"  |"INT_RATE"  |"PURPOSE"  |"REVOL_BAL"  |"PREDICTION"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.0              |71.3          |0.0               |647.0   |10.8365174        |0.0            |0.0        |4409.958333          |75.73          |7.27   |0.0               |0.145       |0.0        |6417.0       |0.0           |
|0.0              |88.5          |4.0               |667.0   |10.79957558       |1.0

In [144]:
# Display a comparison between the actual labels and the predictions made by the model for the test dataset.

predictions_df.select("NOT_FULLY_PAID", "PREDICTION").show()

-----------------------------------
|"NOT_FULLY_PAID"  |"PREDICTION"  |
-----------------------------------
|0.0               |0.0           |
|0.0               |1.0           |
|0.0               |0.0           |
|0.0               |0.0           |
|0.0               |0.0           |
|0.0               |1.0           |
|0.0               |0.0           |
|0.0               |1.0           |
|0.0               |0.0           |
|0.0               |1.0           |
-----------------------------------



In [145]:


score = grid_search.score(test_df)

# Print the score
print(f"Prediction score: {score}")


Prediction score: 0.9016089108910891


In [146]:
FEATURE_COLS=["PURPOSE","CREDIT_POLICY","INT_RATE","INSTALLMENT","LOG_ANNUAL_INC","DTI","FICO","DAYS_WITH_CR_LINE","REVOL_BAL","REVOL_UTIL","INQ_LAST_6MTHS","DELINQ_2YRS","PUB_REC"]
LABEL_COLS=["NOT_FULLY_PAID"]
OUTPUT_COLS=["PREDICTION"]

In [147]:
# Evaluate the model's performance on the test dataset using the scoring metric defined in the grid search, and print the score.
# Calculate and display the training accuracy and confusion matrix for the model using actual and predicted labels from `predictions_df`.

from snowflake.ml.modeling.metrics import accuracy_score, confusion_matrix
traning_accuracy_score=accuracy_score(
    df=predictions_df,
    y_true_col_names=LABEL_COLS,
    y_pred_col_names=OUTPUT_COLS
)
traning_accuracy_score

0.901609

The printed score of 0.907797 indicates that the model accurately predicted the correct outcome for approximately 90.78% of the cases in the test dataset. This high accuracy score suggests that the model is highly effective in making predictions for this particular task.

In [148]:
traning_confusion_matrix=confusion_matrix(
    df=predictions_df,
    y_true_col_name=LABEL_COLS[0],
    y_pred_col_name=OUTPUT_COLS[0]
)
traning_confusion_matrix

array([[693., 108.],
       [ 51., 764.]])

The confusion matrix output represents the performance of the classification model:
<br><br>
Top-left (694): True Negatives (TN) - The number of instances correctly predicted as negative (not fully paid).<br>
Top-right (107): False Positives (FP) - The number of instances incorrectly predicted as positive (fully paid) when they are actually negative.<br>
Bottom-left (42): False Negatives (FN) - The number of instances incorrectly predicted as negative when they are actually positive.<br>
Bottom-right (773): True Positives (TP) - The number of instances correctly predicted as positive.<br>
This matrix helps in understanding the model's ability to correctly or incorrectly classify instances into 'fully paid' or 'not fully paid' categories.<br>

# Model Management and Deployment with Model Registry

with Snowpark ML's model registry, we have a Snowflake native model versioning and deployment framework. This allows us to log models, tag parameters and metrics, track metadata, create versions, and ultimately execute batch inference tasks in a Snowflake warehouse or deploy to a Snowpark Container Service.

In [149]:
# Initialize the Registry class from Snowflake's ML package to manage machine learning models within Snowflake. 
# The registry is set up with the current session's database and schema, allowing for storing, versioning, and deploying ML models directly within Snowflake.

from snowflake.ml.registry import Registry
reg=Registry(session=session,  database_name=session.get_current_database(),schema_name=session.get_current_schema())


In [150]:
# Log the XGBoost model to the Snowflake ML Registry with specified details, including model name, version, requirements, and comments.
# Additionally, include model performance metrics (accuracy score and confusion matrix) and a sample of input data for reference or later use.

mv=reg.log_model(
    grid_search,
    model_name="xgb_loan_default_prediction",
    version_name="v3",
    pip_requirements=["packaging"],
    comment="simple  XGB model to predict loan default",
    metrics={
    "training_accuracy_score":traning_accuracy_score,
    "training_confution_matrix":traning_confusion_matrix.tolist()
    },
    sample_input_data=train_df
)

In [151]:
# Retrieve a list of all models stored in the Snowflake ML Registry within the current session's database and schema, and display the first few entries.

models_df=reg.show_models()
models_df.head()

,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-02-14 13:59:31.924000-08:00,DIAMONDS_PRICE_PREDICTION,ML_HOL_DB,ML_HOL_SCHEMA,None,ACCOUNTADMIN,V0,"[""V0"",""V1"",""V2"",""V3"",""V4"",""V5""]"
1,2024-02-15 07:39:05.441000-08:00,LOAN_DEFAULT_PREDICTION,ML_HOL_DB,ML_HOL_SCHEMA,None,ACCOUNTADMIN,V0,"[""V0"",""V1"",""V2"",""V6""]"
2,2024-02-18 20:40:26.025000-08:00,XGB_LOAN_DEFAULT_PREDICTION,ML_HOL_DB,ML_HOL_SCHEMA,None,ACCOUNTADMIN,V1,"[""V1"",""V2"",""V3""]"


In [152]:
model_list=reg.models()
[m.name for m in model_list]

['DIAMONDS_PRICE_PREDICTION',
 'LOAN_DEFAULT_PREDICTION',
 'XGB_LOAN_DEFAULT_PREDICTION']

In [153]:
rf_model=model_list[2]
rf_model.show_versions()

,created_on,name,comment,database_name,schema_name,module_name,is_default_version,functions,metadata,user_data
0,2024-02-18 20:40:26.049000-08:00,V1,simple XGB model to predict loan default,ML_HOL_DB,ML_HOL_SCHEMA,XGB_LOAN_DEFAULT_PREDICTION,true,"[""PREDICT_PROBA"",""PREDICT""]","{""metrics"": {""training_accuracy_score"": 0.9090...","{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."
1,2024-02-18 20:49:46.047000-08:00,V2,simple XGB model to predict loan default,ML_HOL_DB,ML_HOL_SCHEMA,XGB_LOAN_DEFAULT_PREDICTION,false,"[""PREDICT_PROBA"",""PREDICT""]","{""metrics"": {""training_accuracy_score"": 0.9077...","{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."
2,2024-02-19 12:18:14.312000-08:00,V3,simple XGB model to predict loan default,ML_HOL_DB,ML_HOL_SCHEMA,XGB_LOAN_DEFAULT_PREDICTION,false,"[""PREDICT_PROBA"",""PREDICT""]","{""metrics"": {""training_accuracy_score"": 0.9016...","{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."


In [154]:
# Load version 3 of the third model from the retrieved model list, run it to predict on the test dataset using the 'predict' function, 
# and then display the actual labels alongside the predictions for comparison.

rf_mv1=model_list[2].version('V3')
pred_df1=rf_mv1.run(test_df,function_name='predict')
pred_df1['NOT_FULLY_PAID','PREDICTION'].show()

-----------------------------------
|"NOT_FULLY_PAID"  |"PREDICTION"  |
-----------------------------------
|0.0               |0.0           |
|0.0               |1.0           |
|0.0               |0.0           |
|0.0               |0.0           |
|0.0               |0.0           |
|0.0               |1.0           |
|0.0               |0.0           |
|0.0               |1.0           |
|0.0               |0.0           |
|0.0               |1.0           |
-----------------------------------

